In [1]:
# %conda install -c conda-forge ta-lib -q
# %pip install -r requirements.txt -q

# import boto3
import data
import glob
import matplotlib.pyplot as plt
import models
import numpy as np
import os
import pandas as pd
import pypickle
import strategy
import time

from data import * 
from feature_engineering import FeaturesEngineering
from models import Models
from io import StringIO
from IPython.display import display
from sklearn.linear_model import LassoCV
from utils import log

2025-03-01 18:15:20.856636: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-01 18:15:20.866864: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1740824120.879122   26461 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1740824120.882733   26461 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-01 18:15:20.895664: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [2]:
pypickle_overwrite = True

models = Models(use_xgboost_gpu=True)

In [3]:
dir_project = os.getcwd()
dir_models = os.path.join(dir_project, 'models')
dir_models_base = os.path.join(dir_project, 'models', 'base')
dir_models_scaler = os.path.join(dir_project, 'models', 'scaler')
dir_models_best = os.path.join(dir_project, 'models', 'best')

print(f"""
Project directory: {dir_project}
Models directory: {dir_models}
Base models directory: {dir_models_base}
Scalers directory: {dir_models_scaler}
Best models directory: {dir_models_best}
""")


Project directory: /home/alfred/projects/mscfe-masters-project/src
Models directory: /home/alfred/projects/mscfe-masters-project/src/models
Base models directory: /home/alfred/projects/mscfe-masters-project/src/models/base
Scalers directory: /home/alfred/projects/mscfe-masters-project/src/models/scaler
Best models directory: /home/alfred/projects/mscfe-masters-project/src/models/best



# Read raw data

In [4]:
# Define S3 bucket and file path
bucket_name = 'mscfe-capstone-project-bucket'
file_key = 'filtered_data.csv'

filtered = pd.read_csv('filtered_data.csv').drop(['Unnamed: 0'], axis=1)
display(filtered)

,seriesX,seriesY,correlation,spread_adf_stat,spread_adf_pValue,spread_stationary,series_coint_stat,series_coint_pValue,series_cointegrated
0,GOOG,GOOGL,0.999931,-4.971937,0.000025,True,-4.976339,0.000182,True
1,META,DAXEX_ETF,0.888934,-2.917056,0.043400,True,-3.483091,0.033924,True
2,AMZN,BNBUSDT,0.877059,-3.631203,0.005192,True,-3.545100,0.028601,True
3,AMZN,TRXUSDT,0.814577,-3.869803,0.002269,True,-3.387413,0.043768,True
4,SPY_ETF,AVAXUSDT,0.847292,-3.309927,0.014441,True,-3.698512,0.018406,True
5,SPY_ETF,ETHUSDT,0.862483,-3.428205,0.010034,True,-3.737367,0.016394,True
6,EQQQ_ETF,BNBUSDT,0.886060,-3.816404,0.002744,True,-3.670509,0.019987,True
7,BNBUSDT,ADAUSDT,0.901459,-3.279504,0.015820,True,-3.775562,0.014607,True
8,BNBUSDT,BTCUSDT,0.884604,-2.909961,0.044202,True,-3.393160,0.043116,True
9,BNBUSDT,TRXUSDT,0.933562,-3.656380,0.004770,True,-4.003675,0.007095,True


In [5]:
print('********************')
pairs = filtered['seriesX'].iloc[1:] + ' ' + filtered['seriesY'].iloc[1:]
print('Trading pairs:')
for p in pairs:
    print(p)
print('********************')

unique_tickers = pd.concat([filtered['seriesX'].iloc[1:], filtered['seriesY'].iloc[1:]]).drop_duplicates()
print('Unique tickers:')
for ut in unique_tickers:
    print(ut)
print('********************')

# Preview the source data
file_key = 'df_full.csv'
df_full = pd.read_csv('df_full.csv').drop(['Unnamed: 0'], axis=1)
# obj = s3.get_object(Bucket=bucket_name, Key=file_key)
# df_full = pd.read_csv(StringIO(obj['Body'].read().decode('utf-8'))).drop(['Unnamed: 0'], axis=1)
display(df_full.sample(5))

********************
Trading pairs:
META DAXEX_ETF
AMZN BNBUSDT
AMZN TRXUSDT
SPY_ETF AVAXUSDT
SPY_ETF ETHUSDT
EQQQ_ETF BNBUSDT
BNBUSDT ADAUSDT
BNBUSDT BTCUSDT
BNBUSDT TRXUSDT
BNBUSDT XRPUSDT
********************
Unique tickers:
META
AMZN
SPY_ETF
EQQQ_ETF
BNBUSDT
DAXEX_ETF
TRXUSDT
AVAXUSDT
ETHUSDT
ADAUSDT
BTCUSDT
XRPUSDT
********************


,OpenTime,Open,High,Low,Close,Symbol
172973,2024-11-05 06:15:00,0.1614,0.1614,0.1612,0.1613,TRXUSDT
51041,2024-12-11 18:05:00,608.0200,608.0400,607.8700,607.9400,SPY_ETF
21104,2024-11-07 17:45:00,592.6950,593.3999,592.5600,592.9850,META
253554,2024-12-03 05:45:00,0.9999,1.0000,0.9998,0.9998,USDCUSDT
75397,2025-01-13 12:25:00,167.1200,167.1200,167.0400,167.1000,DAXEX_ETF


## Placeholders

In [6]:
dict_data = {}
dict_cv_features = {}

# Features Engineering

## Create and normalizing features

In [7]:
# Initializing the FeaturesEngineering obj
feat = FeaturesEngineering()

# Default config
feat.default_config

{'ema': [8, 21, 55],
 'macd': {'fast': 12, 'slow': 26, 'signal': 9},
 'rsi': [14],
 'bb': {'timeperiod': 20, 'nbdevup': 2, 'nbdevdn': 2},
 'atr': {'timeperiod': 14},
 'stoch': {'fastk_period': 14, 'slowk_period': 3, 'slowd_period': 3},
 'cci': {'timeperiod': 20},
 'willr': {'timeperiod': 14}}

1. Create features from pairs data
2. Load saved scaler
3. Perform normalization / standardization

In [8]:
scaler_type = "StandardScaler"
dict_data['df_feat'] = {}
dict_data['df_norm'] = {}
dict_data['scaler'] = {}
i = 1

for p in pairs:

    print(f"({i}) Pairs: {p}")
    tickerX, tickerY = p.split(' ')
    
    # 1. Create features from pairs data
    df_feat = feat.create_features(
        tickerX, 
        tickerY,
        df_full[df_full['Symbol']==tickerX],
        df_full[df_full['Symbol']==tickerY],
        config=None, # using default config in the FeaturesEngineering
        dropna=True
    )

    # 2. Load saved scaler
    scaler_name = f"{scaler_type.lower()}_{p.replace(' ', '_')}.pkl"
    dir_scaler = os.path.join(dir_models_scaler, scaler_name)
    scaler = pypickle.load(dir_scaler)

    # 3. Perform normalization / standardization
    data_scaled = scaler.transform(df_feat)
    df_norm = pd.DataFrame(
        data_scaled, index=df_feat.index, columns=df_feat.columns
    )
    df_norm = df_norm.rename(columns={'Spread':'NormalizedSpread'})
    
    dict_data['df_feat'][p] = df_feat
    dict_data['df_norm'][p] = df_norm
    dict_data['scaler'][p] = scaler

    i+=1
    print('')

(1) Pairs: META DAXEX_ETF
[pypickle] Pickle file loaded: [/home/alfred/projects/mscfe-masters-project/src/models/scaler/standardscaler_META_DAXEX_ETF.pkl]

(2) Pairs: AMZN BNBUSDT
[pypickle] Pickle file loaded: [/home/alfred/projects/mscfe-masters-project/src/models/scaler/standardscaler_AMZN_BNBUSDT.pkl]

(3) Pairs: AMZN TRXUSDT
[pypickle] Pickle file loaded: [/home/alfred/projects/mscfe-masters-project/src/models/scaler/standardscaler_AMZN_TRXUSDT.pkl]

(4) Pairs: SPY_ETF AVAXUSDT
[pypickle] Pickle file loaded: [/home/alfred/projects/mscfe-masters-project/src/models/scaler/standardscaler_SPY_ETF_AVAXUSDT.pkl]

(5) Pairs: SPY_ETF ETHUSDT
[pypickle] Pickle file loaded: [/home/alfred/projects/mscfe-masters-project/src/models/scaler/standardscaler_SPY_ETF_ETHUSDT.pkl]

(6) Pairs: EQQQ_ETF BNBUSDT
[pypickle] Pickle file loaded: [/home/alfred/projects/mscfe-masters-project/src/models/scaler/standardscaler_EQQQ_ETF_BNBUSDT.pkl]

(7) Pairs: BNBUSDT ADAUSDT
[pypickle] Pickle file loaded: [/ho

## Read LassoCV selected features

In [9]:
lasso_cv_name = f"lasso_cv_features.pkl"
dir_lasso = os.path.join(dir_models, lasso_cv_name)
dict_cv_features = pypickle.load(dir_lasso)

for key in dict_cv_features:
    print(f"{key}: {dict_cv_features[key]}")

[pypickle] Pickle file loaded: [/home/alfred/projects/mscfe-masters-project/src/models/lasso_cv_features.pkl]
META DAXEX_ETF: ['META_EMA8', 'META_EMA55', 'META_RSI14', 'META_ATR14', 'META_StochD3', 'META_CCI20', 'META_WILLR14', 'DAXEX_ETF_EMA8', 'DAXEX_ETF_EMA21', 'DAXEX_ETF_MACD', 'DAXEX_ETF_BBupper', 'DAXEX_ETF_BBmiddle', 'DAXEX_ETF_BBlower', 'DAXEX_ETF_StochD3']
AMZN BNBUSDT: ['AMZN_EMA8', 'AMZN_EMA21', 'AMZN_EMA55', 'AMZN_RSI14', 'AMZN_ATR14', 'AMZN_StochD3', 'AMZN_WILLR14', 'BNBUSDT_EMA8', 'BNBUSDT_EMA55', 'BNBUSDT_MACD', 'BNBUSDT_RSI14', 'BNBUSDT_ATR14', 'BNBUSDT_StochD3', 'BNBUSDT_CCI20', 'BNBUSDT_WILLR14']
AMZN TRXUSDT: ['AMZN_EMA8', 'AMZN_EMA55', 'AMZN_RSI14', 'AMZN_ATR14', 'AMZN_StochD3', 'AMZN_CCI20', 'AMZN_WILLR14', 'TRXUSDT_MACD']
SPY_ETF AVAXUSDT: ['SPY_ETF_EMA8', 'SPY_ETF_EMA55', 'SPY_ETF_MACD', 'SPY_ETF_RSI14', 'SPY_ETF_BBupper', 'SPY_ETF_BBmiddle', 'SPY_ETF_BBlower', 'SPY_ETF_ATR14', 'SPY_ETF_StochK14', 'SPY_ETF_StochD3', 'SPY_ETF_CCI20', 'SPY_ETF_WILLR14', 'AVAXUSDT_E

# Hyperparameters Tunning

## Transformer Regression

In [10]:
dir_best = os.path.join(dir_models_best, 'transformer')
param_grid = {
    'num_heads': [2, 4],
    'dropout_rate': [0.25],  # Dropout rates
    'batch_size': [32, 64], # Number of data per batch
    'ff_dim': [64, 128],
}

verbose = True
tuned_params = []
i = 1

for p in pairs:
    print(f"({i}) Pairs: {p}")

    # 1. Read normalized features
    df_norm = dict_data['df_norm'][p]
    
    # 2. Create X, y based on LassoCV
    lasso_features_cols = dict_cv_features[p]
    target_col = df_norm.columns.tolist()[-1]
    df_lasso = pd.concat([df_norm[lasso_features_cols], df_norm[target_col]], axis=1)

    # 3. Run Hyperparameters tunning
    start_time = time.perf_counter()
    best_model, best_params, best_mse, best_df_test = models.hyperparameter_tuning(
        df=df_lasso, p=p, model_func=models.transformer_regression, param_grid=param_grid, verbose=verbose)
    end_time = time.perf_counter()
    elapsed_time = end_time - start_time

    stats = {
        'pair':p, 
        'best_mse': best_mse,
        'data_points': len(df_lasso),
        'features': len(df_lasso.columns),
    }
    stats.update(best_params)
    tuned_params.append(stats)

    print(f"Best MSE: {best_mse:.6f}")
    print(f"num_heads: {best_params['num_heads']}, ff_dim: {best_params['ff_dim']}, batch_size: {best_params['batch_size']}")
    print(f"Time taken: {elapsed_time:.6f} seconds")

    # 4. Save best model future use like testing and strategy
    model_name = f"best_transformer_{p.replace(' ', '_')}.pkl"
    dir_model_name = os.path.join(dir_best, model_name)
    pypickle.save(dir_model_name, best_model, overwrite=pypickle_overwrite)

    i+=1
    print('')

dir_param = os.path.join(dir_best, 'best_model_results.csv')
df_transformer = pd.DataFrame(tuned_params).set_index('pair')
df_transformer.to_csv(dir_param)
display(df_transformer)

(1) Pairs: META DAXEX_ETF
Testing params: {'num_heads': 2, 'dropout_rate': 0.25, 'batch_size': 32, 'ff_dim': 64}
Training/Validation Phase


I0000 00:00:1740824123.312719   26461 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 21799 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4090, pci bus id: 0000:01:00.0, compute capability: 8.9
I0000 00:00:1740824125.563107   26612 service.cc:148] XLA service 0x75f7c8005990 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1740824125.563130   26612 service.cc:156]   StreamExecutor device (0): NVIDIA GeForce RTX 4090, Compute Capability 8.9
2025-03-01 18:15:25.608526: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1740824125.820259   26612 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1740824128.740824   26612 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 107ms/step
Transformer MSE: 0.040626238278947575
Testing params: {'num_heads': 2, 'dropout_rate': 0.25, 'batch_size': 32, 'ff_dim': 128}
Training/Validation Phase
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 124ms/step
Transformer MSE: 0.04928884863831479
Testing params: {'num_heads': 2, 'dropout_rate': 0.25, 'batch_size': 64, 'ff_dim': 64}
Training/Validation Phase


2025-03-01 18:15:53.577154: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'input_multiply_reduce_select_fusion', 8 bytes spill stores, 4 bytes spill loads
ptxas warning : Registers are spilled to local memory in function 'input_add_multiply_reduce_select_fusion', 8 bytes spill stores, 4 bytes spill loads



8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 48ms/step 
Transformer MSE: 0.04657388803518155
Testing params: {'num_heads': 2, 'dropout_rate': 0.25, 'batch_size': 64, 'ff_dim': 128}
Training/Validation Phase


...

2025-03-01 19:01:32.095523: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'input_add_multiply_reduce_select_fusion', 56 bytes spill stores, 44 bytes spill loads



194/194 ━━━━━━━━━━━━━━━━━━━━ 0s 569us/step  
Transformer MSE: 0.031239750589565452
Testing params: {'num_heads': 4, 'dropout_rate': 0.25, 'batch_size': 64, 'ff_dim': 128}
Training/Validation Phase


2025-03-01 19:02:03.434951: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'input_add_multiply_reduce_select_fusion', 56 bytes spill stores, 44 bytes spill loads



194/194 ━━━━━━━━━━━━━━━━━━━━ 0s 583us/step  
Transformer MSE: 0.018798797823860555
Best parameters: {'num_heads': 2, 'dropout_rate': 0.25, 'batch_size': 64, 'ff_dim': 64}, Best MSE: 0.0035076021076534884
Best MSE: 0.003508
num_heads: 2, ff_dim: 64, batch_size: 64
Time taken: 347.606255 seconds
[pypickle] Pickle file saved: [/home/alfred/projects/mscfe-masters-project/src/models/best/transformer/best_transformer_BNBUSDT_XRPUSDT.pkl]



,best_mse,data_points,features,num_heads,dropout_rate,batch_size,ff_dim
pair,,,,,,,
META DAXEX_ETF,0.040626,1145,15,2,0.25,32,64
AMZN BNBUSDT,0.012462,4991,16,2,0.25,64,64
AMZN TRXUSDT,0.003325,4991,9,2,0.25,64,128
SPY_ETF AVAXUSDT,0.044844,4997,21,2,0.25,64,64
SPY_ETF ETHUSDT,0.008839,4997,16,4,0.25,64,64
EQQQ_ETF BNBUSDT,0.025240,3217,13,2,0.25,64,64
BNBUSDT ADAUSDT,0.000998,31051,9,4,0.25,64,128
BNBUSDT BTCUSDT,0.001984,31051,9,2,0.25,32,64
BNBUSDT TRXUSDT,0.001316,31051,12,4,0.25,32,64
